## Supervised Machine Learning - Logistic Regression Algorithm (Galaxies vs. Stars vs. QSO)

### Dependencies:

In [162]:
import numpy as np
import pandas as pd

from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract

In [163]:
import sqlite3

# Create db connection to sqlite3 and cursor to execute queries
conn = sqlite3.connect("stellar_class_db")
c = conn.cursor()



In [164]:
# Load the data from sqlite table "stellar"
data_df = pd.read_sql_query("SELECT * FROM stellar", conn)
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,mjd,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,1.055431e+19,GALAXY,0.000000,9374,57749,438
99996,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,8.586351e+18,GALAXY,0.404895,7626,56934,866
99997,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,3.112008e+18,GALAXY,0.143366,2764,54535,74
99998,1.237661e+18,212.268621,46.660365,25.35039,21.63757,19.91386,19.07254,18.62482,3650,301,4,131,7.601080e+18,GALAXY,0.455040,6751,56368,470


In [165]:
data_df.set_index(['spec_obj_ID'], inplace = True)

In [166]:
data_df

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,class,redshift,plate,mjd,fiber_ID
spec_obj_ID,,,,,,,,,,,,,,,,,
6.543777e+18,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,GALAXY,0.634794,5812,56354,171
1.176014e+19,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,GALAXY,0.779136,10445,58158,427
5.152200e+18,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,GALAXY,0.644195,4576,55592,299
1.030107e+19,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,GALAXY,0.932346,9149,58039,775
6.891865e+18,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,GALAXY,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.055431e+19,1.237679e+18,39.620709,-2.594074,22.16759,22.97586,21.90404,21.30548,20.73569,7778,301,2,581,GALAXY,0.000000,9374,57749,438
8.586351e+18,1.237679e+18,29.493819,19.798874,22.69118,22.38628,20.45003,19.75759,19.41526,7917,301,1,289,GALAXY,0.404895,7626,56934,866
3.112008e+18,1.237668e+18,224.587407,15.700707,21.16916,19.26997,18.20428,17.69034,17.35221,5314,301,4,308,GALAXY,0.143366,2764,54535,74


In [167]:
photo_df = data_df[['alpha','delta','run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'mjd', 'fiber_ID']] 
photo_df

,alpha,delta,run_ID,rerun_ID,cam_col,field_ID,obj_ID,plate,mjd,fiber_ID
spec_obj_ID,,,,,,,,,,
6.543777e+18,135.689107,32.494632,3606,301,2,79,1.237661e+18,5812,56354,171
1.176014e+19,144.826101,31.274185,4518,301,5,119,1.237665e+18,10445,58158,427
5.152200e+18,142.188790,35.582444,3606,301,2,120,1.237661e+18,4576,55592,299
1.030107e+19,338.741038,-0.402828,4192,301,3,214,1.237663e+18,9149,58039,775
6.891865e+18,345.282593,21.183866,8102,301,3,137,1.237680e+18,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...
1.055431e+19,39.620709,-2.594074,7778,301,2,581,1.237679e+18,9374,57749,438
8.586351e+18,29.493819,19.798874,7917,301,1,289,1.237679e+18,7626,56934,866
3.112008e+18,224.587407,15.700707,5314,301,4,308,1.237668e+18,2764,54535,74


In [168]:
 spectral_df=data_df.drop(['alpha','delta','run_ID','rerun_ID', 'cam_col', 'field_ID', 'plate', 'mjd', 'fiber_ID'], axis = 1 )

In [169]:
spectral_df

,obj_ID,u,g,r,i,z,class,redshift
spec_obj_ID,,,,,,,,
6.543777e+18,1.237661e+18,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1.176014e+19,1.237665e+18,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
5.152200e+18,1.237661e+18,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
1.030107e+19,1.237663e+18,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
6.891865e+18,1.237680e+18,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123
...,...,...,...,...,...,...,...,...
1.055431e+19,1.237679e+18,22.16759,22.97586,21.90404,21.30548,20.73569,GALAXY,0.000000
8.586351e+18,1.237679e+18,22.69118,22.38628,20.45003,19.75759,19.41526,GALAXY,0.404895
3.112008e+18,1.237668e+18,21.16916,19.26997,18.20428,17.69034,17.35221,GALAXY,0.143366


In [170]:
# Write photometric features to new SQL table photo_properties 

photo_df.to_sql("photo_properties", conn, if_exists="replace")



In [171]:
# Write spectral characteric features to new SQL table spectral_characteristics 

spectral_df.to_sql("spectral_characteristics", conn, if_exists="replace")

conn.close()

In [187]:
conn = sqlite3.connect("stellar_class_db")
full_data = pd.read_sql_query("SELECT * FROM photo_properties JOIN spectral_characteristics ON photo_properties.spec_obj_ID=spectral_characteristics.spec_obj_ID", conn)

In [188]:
full_data

,spec_obj_ID,alpha,delta,run_ID,rerun_ID,cam_col,field_ID,obj_ID,plate,mjd,fiber_ID,spec_obj_ID,obj_ID,u,g,r,i,z,class,redshift
0,6.543777e+18,135.689107,32.494632,3606,301,2,79,1.237661e+18,5812,56354,171,6.543777e+18,1.237661e+18,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1,1.176014e+19,144.826101,31.274185,4518,301,5,119,1.237665e+18,10445,58158,427,1.176014e+19,1.237665e+18,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
2,5.152200e+18,142.188790,35.582444,3606,301,2,120,1.237661e+18,4576,55592,299,5.152200e+18,1.237661e+18,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
3,1.030107e+19,338.741038,-0.402828,4192,301,3,214,1.237663e+18,9149,58039,775,1.030107e+19,1.237663e+18,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
4,6.891865e+18,345.282593,21.183866,8102,301,3,137,1.237680e+18,6121,56187,842,6.891865e+18,1.237680e+18,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.055431e+19,39.620709,-2.594074,7778,301,2,581,1.237679e+18,9374,57749,438,1.055431e+19,1.237679e+18,22.16759,22.97586,21.90404,21.30548,20.73569,GALAXY,0.000000
99996,8.586351e+18,29.493819,19.798874,7917,301,1,289,1.237679e+18,7626,56934,866,8.586351e+18,1.237679e+18,22.69118,22.38628,20.45003,19.75759,19.41526,GALAXY,0.404895
99997,3.112008e+18,224.587407,15.700707,5314,301,4,308,1.237668e+18,2764,54535,74,3.112008e+18,1.237668e+18,21.16916,19.26997,18.20428,17.69034,17.35221,GALAXY,0.143366
99998,7.601080e+18,212.268621,46.660365,3650,301,4,131,1.237661e+18,6751,56368,470,7.601080e+18,1.237661e+18,25.35039,21.63757,19.91386,19.07254,18.62482,GALAXY,0.455040


In [189]:
new_data_df = full_data.loc[:,~full_data.columns.duplicated()]

In [190]:
new_data_df

,spec_obj_ID,alpha,delta,run_ID,rerun_ID,cam_col,field_ID,obj_ID,plate,mjd,fiber_ID,u,g,r,i,z,class,redshift
0,6.543777e+18,135.689107,32.494632,3606,301,2,79,1.237661e+18,5812,56354,171,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1,1.176014e+19,144.826101,31.274185,4518,301,5,119,1.237665e+18,10445,58158,427,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
2,5.152200e+18,142.188790,35.582444,3606,301,2,120,1.237661e+18,4576,55592,299,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
3,1.030107e+19,338.741038,-0.402828,4192,301,3,214,1.237663e+18,9149,58039,775,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
4,6.891865e+18,345.282593,21.183866,8102,301,3,137,1.237680e+18,6121,56187,842,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1.055431e+19,39.620709,-2.594074,7778,301,2,581,1.237679e+18,9374,57749,438,22.16759,22.97586,21.90404,21.30548,20.73569,GALAXY,0.000000
99996,8.586351e+18,29.493819,19.798874,7917,301,1,289,1.237679e+18,7626,56934,866,22.69118,22.38628,20.45003,19.75759,19.41526,GALAXY,0.404895
99997,3.112008e+18,224.587407,15.700707,5314,301,4,308,1.237668e+18,2764,54535,74,21.16916,19.26997,18.20428,17.69034,17.35221,GALAXY,0.143366
99998,7.601080e+18,212.268621,46.660365,3650,301,4,131,1.237661e+18,6751,56368,470,25.35039,21.63757,19.91386,19.07254,18.62482,GALAXY,0.455040


In [191]:
new_data_df.set_index(['spec_obj_ID'], inplace = True)

### Variable engineering

In [192]:
frequency = [1, 2, 3]

u_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_u"] = pd.cut(new_data_df["u"], u_size_bins, labels = frequency)

g_size_bins = [-10000, -6600,-3310, 33]
new_data_df["binned_g"] = pd.cut(new_data_df["g"], g_size_bins, labels = frequency)

r_size_bins = [9,16,23, 30]
new_data_df["binned_r"] = pd.cut(new_data_df["r"], r_size_bins, labels = frequency)

i_size_bins = [9,17,25, 33]
new_data_df["binned_i"] = pd.cut(new_data_df["i"], i_size_bins, labels = frequency)

z_size_bins = [-10000, -6600,-3310, 30]
new_data_df["binned_z"] = pd.cut(new_data_df["z"], z_size_bins, labels = frequency)

new_data_df.sample(30)

C:\Users\c_tal\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\c_tal\anaconda3\envs\PythonData\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,alpha,delta,run_ID,rerun_ID,cam_col,field_ID,obj_ID,plate,mjd,fiber_ID,...,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,,,,,,,,,,
7.881987e+17,27.810853,-1.187841,4263,301,1,301,1.237664e+18,700,52199,250,...,18.63856,18.03709,17.74314,GALAXY,0.242836,3,3,2,2,3
7.535728e+18,166.009243,47.026535,3530,301,6,301,1.237661e+18,6693,56384,290,...,21.27173,21.35394,20.90903,QSO,2.831713,3,3,2,2,3
1.178823e+19,161.544968,27.663409,5061,301,3,311,1.237667e+18,10470,58130,194,...,21.51353,21.03254,20.70683,GALAXY,0.817741,3,3,2,2,3
1.431119e+18,133.544668,32.904070,3606,301,5,70,1.237661e+18,1271,52974,366,...,20.11540,19.98673,19.79300,QSO,2.052829,3,3,2,2,3
4.348285e+18,211.523258,31.285576,4576,301,2,598,1.237665e+18,3862,55276,215,...,21.04583,20.09291,19.80134,GALAXY,0.502024,3,3,2,2,3
7.745066e+18,2.471481,22.665218,8108,301,4,257,1.237680e+18,6879,56539,1,...,21.12471,20.00920,19.69624,GALAXY,0.505212,3,3,2,2,3
5.900637e+17,195.313868,3.263423,1458,301,5,471,1.237652e+18,524,52027,335,...,16.63568,16.19748,15.87807,GALAXY,0.099134,3,3,2,1,3
8.534422e+18,330.225931,22.529550,8110,301,2,47,1.237680e+18,7580,56955,367,...,21.23280,20.10739,19.41960,STAR,0.000019,3,3,2,2,3
5.450268e+17,136.445921,59.106625,1412,301,5,18,1.237652e+18,484,51907,332,...,18.67504,18.66854,18.67316,QSO,1.143496,3,3,2,2,3


In [193]:
new_data_df = new_data_df.drop(['alpha','delta','run_ID','rerun_ID', 'cam_col', 'field_ID', "obj_ID",'plate', 'mjd', 'fiber_ID'], axis = 1)
new_data_df

,u,g,r,i,z,class,redshift,binned_u,binned_g,binned_r,binned_i,binned_z
spec_obj_ID,,,,,,,,,,,,
6.543777e+18,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794,3,3,2,2,3
1.176014e+19,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136,3,3,2,2,3
5.152200e+18,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195,3,3,2,2,3
1.030107e+19,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346,3,3,2,2,3
6.891865e+18,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123,3,3,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1.055431e+19,22.16759,22.97586,21.90404,21.30548,20.73569,GALAXY,0.000000,3,3,2,2,3
8.586351e+18,22.69118,22.38628,20.45003,19.75759,19.41526,GALAXY,0.404895,3,3,2,2,3
3.112008e+18,21.16916,19.26997,18.20428,17.69034,17.35221,GALAXY,0.143366,3,3,2,2,3


In [180]:
new_data_df.iloc[new_data_df.values==np.inf]
new_data_df =new_data_df[~new_data_df.isin([np.nan, np.inf, -np.inf]).any(1)]

### Test and Train Data

In [194]:
# Create our features
X = new_data_df.drop('class', axis=1)

# Create our target
y = new_data_df['class']

print(X.shape)
print(y.shape)
Counter(y)

(100000, 11)
(100000,)


Counter({'GALAXY': 59445, 'QSO': 18961, 'STAR': 21594})

In [195]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'STAR': 16227, 'GALAXY': 44608, 'QSO': 14165})

### Random Oversampling

In [196]:
# Resample the training data with the RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'STAR': 44608, 'GALAXY': 44608, 'QSO': 44608})

### Logestic Regression (Galaxies, QSO and Stars) - Oversampling

In [197]:
# Train the Logestic Regression Model using the resampled data
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='none', random_state=1, max_iter=100000)
model.fit(X_resampled, y_resampled)

LogisticRegression(max_iter=100000, multi_class='multinomial', penalty='none',
                   random_state=1)

In [198]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9603254168553125

In [199]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       0.98      0.95      0.97      0.96      0.96      0.92     14837
        QSO       0.88      0.93      0.97      0.90      0.95      0.90      4796
       STAR       0.98      1.00      0.99      0.99      1.00      0.99      5367

avg / total       0.96      0.96      0.97      0.96      0.97      0.93     25000



In [200]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred)
cmros = pd.DataFrame(c, index=["Actual - Galaxy", "Actual - QSO", "Actual - Star"], columns=["Predicted - Galaxy", "Predicted - QSO", "Predicted - Star"])
cmros

,Predicted - Galaxy,Predicted - QSO,Predicted - Star
Actual - Galaxy,14082,630,125
Actual - QSO,325,4471,0
Actual - Star,2,0,5365


### Random Undersampling

In [201]:
# Resample the training data with the RandomUnderSampler
rus = RandomUnderSampler(random_state=1)
X_resampled1, y_resampled1 = rus.fit_resample(X_train, y_train)
Counter(y_resampled1)

Counter({'GALAXY': 14165, 'QSO': 14165, 'STAR': 14165})

### Logistic Regression (Galaxies, QSO and Stars) - Undersampling

In [202]:
model2 = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='none', random_state=1, max_iter=100000)
model2.fit(X_resampled1, y_resampled1)

LogisticRegression(max_iter=100000, multi_class='multinomial', penalty='none',
                   random_state=1)

In [203]:
# Calculated the balanced accuracy score
y_pred1 = model2.predict(X_test)
balanced_accuracy_score(y_test, y_pred1)

0.9606462557277081

In [204]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test,y_pred1))

                   pre       rec       spe        f1       geo       iba       sup

     GALAXY       0.98      0.95      0.97      0.96      0.96      0.92     14837
        QSO       0.88      0.93      0.97      0.90      0.95      0.90      4796
       STAR       0.98      1.00      0.99      0.99      1.00      0.99      5367

avg / total       0.96      0.96      0.97      0.96      0.97      0.93     25000



In [205]:
# Display the confusion matrix
c= confusion_matrix(y_test,y_pred1)
cmrus = pd.DataFrame(c, index=["Actual - Galaxy", "Actual - QSO", "Actual - Star"], columns=["Predicted - Galaxy", "Predicted - QSO", "Predicted - Star"])
cmrus

,Predicted - Galaxy,Predicted - QSO,Predicted - Star
Actual - Galaxy,14087,626,124
Actual - QSO,322,4474,0
Actual - Star,2,0,5365


In [206]:
# Write new_data_df log. reg full data mlm results to new SQL table full_log_r

new_data_df.to_sql("full_log_r", conn, if_exists="replace")

conn.close()

In [209]:
import pandas as pd
from config import password
from sqlalchemy import create_engine
engine = create_engine("postgresql://postgres:" + password + "@localhost:5432/Stellar_Classification")

In [210]:
# Create table with results in postgres 

new_data_df.to_sql("full_log_r", engine, if_exists="replace")
